<a href="https://colab.research.google.com/github/ethvedbitdesjan/SummerResearch/blob/main/SimpleClassifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 390, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 390 (delta 89), reused 51 (delta 51), pack-reused 269
Receiving objects: 100% (390/390), 107.11 KiB | 2.75 MiB/s, done.
Resolving deltas: 100% (191/191), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.7 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.

In [1]:
# Critical imports
import cudf
import cuml
import os
import numpy as np
import pandas as pd
from PIL import Image
import cv2

In [2]:
from google.colab import drive
drive.mount('content/')

Drive already mounted at content/; to attempt to forcibly remount, call drive.mount("content/", force_remount=True).


In [3]:
%cd /content/content/MyDrive/SummerResearch

/content/content/MyDrive/SummerResearch


In [4]:
train_data = []
train_data_label = []
for file in os.listdir('Funny_Originals_Final'):
  img_array = cv2.imread(os.path.join('Funny_Originals_Final', file))
  img_array_resized = cv2.resize(img_array, (150, 150))
  train_data.append((img_array_resized.reshape(-1).astype(np.float64), 1))
  train_data_label.append(1)
for file in os.listdir('Non-Funny_Modified_Final'):
  img_array = cv2.imread(os.path.join('Non-Funny_Modified_Final', file))
  img_array_resized = cv2.resize(img_array, (150, 150))
  train_data.append((img_array_resized.reshape(-1).astype(np.float64), 0))
  train_data_label.append(0)

In [5]:
import random
random.shuffle(train_data)

In [6]:
x = []
y = []
for xi, yi in train_data:
  x.append(xi)
  y.append(yi)
x = np.array(x)
y = np.array(y)

In [15]:
from cuml.svm import SVC
svm = SVC()

In [8]:
x.shape, y.shape

((2636, 67500), (2636,))

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.35)

In [16]:
svm.fit(x_train, y_train)

[W] [21:04:49.330770] Warning: could not fill working set, found only 1021 elements
[W] [21:04:50.122531] Warning: could not fill working set, found only 1007 elements
[W] [21:04:50.900884] Warning: could not fill working set, found only 1019 elements


SVC()

In [17]:
from sklearn.metrics import accuracy_score
y_pred = svm.predict(x_test)
accuracy_score(y_pred, y_test)

0.5850487540628385

Accuracy^

In [22]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_pred, y_test), columns=['Test Not Funny', 'Test Funny'], index=['Pred Not Funny', 'Pred Funny'])

,Test Not Funny,Test Funny
Pred Not Funny,9,32
Pred Funny,351,531


In [10]:
import xgboost as xgb

In [11]:
dtrain = xgb.DMatrix(
        x_train,
        y_train    )

dtest = xgb.DMatrix(
        x_test,
        y_test    )

## Train the model
trained_model = xgb.train(
                        {
                          'learning_rate': 0.1,
                          'colsample_bytree' : 0.3,
                          'max_depth': 5,
                          'objective': 'reg:linear',
                          'n_estimators':10,
                          'alpha' : 10,
                          'silent': True,
                          'verbose_eval': True,
                          'tree_method':'gpu_hist',
                        },
                        dtrain,
                        num_boost_round=100, evals=[(dtrain, 'train')])

## Predict the model
prediction = trained_model.predict(dtest)

[21:02:19] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[21:02:19] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators", "silent", "verbose_eval" } are not used.

[0]	train-rmse:0.49479
[1]	train-rmse:0.49014
[2]	train-rmse:0.48460
[3]	train-rmse:0.48179
[4]	train-rmse:0.47680
[5]	train-rmse:0.47192
[6]	train-rmse:0.46770
[7]	train-rmse:0.46277
[8]	train-rmse:0.45917
[9]	train-rmse:0.45556
[10]	train-rmse:0.45210
[11]	train-rmse:0.44820
[12]	train-rmse:0.44463
[13]	train-rmse:0.44127
[14]	train-rmse:0.43750
[15]	train-rmse:0.43408
[16]	train-rmse:0.43135
[17]	train-rmse:0.42820
[18]	train-rmse:0.42544
[19]	train-rmse:0.42244
[20]	train-rmse:0.41956
[21]	train-rmse:0.41649
[22]	train-rmse:0.41362
[23]	train-rmse:0.41118
[24]	train-rmse:0.40877
[25]	train-rmse:0.40605
[26]	train-rmse:0.40396
[27]	train-rmse:0.40155
[28]	train-rmse:0.39952
[29]	train-rmse:0.39715
[30]	train-rmse:0.39501
[31]	train-rmse:0.39338
[32]	tr

In [12]:
y_test = y_test.astype(np.float64)

## Test prediction wih RMSE, compare it to sklearn and pandas.
from cuml.metrics import mean_squared_error, accuracy_score
rmse = np.sqrt(mean_squared_error(y_test, prediction))

## RMSE for the price prediction is per 1000$.  Let's see what we get...
print("RMSE: ", rmse, "Accuracy: ", accuracy_score(prediction, y_test))

RMSE:  0.5747330044717535 Accuracy:  0.39003250002861023


In [13]:
del trained_model
del prediction